In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
warnings.filterwarnings("ignore")

In [2]:
raw_train = pd.read_csv("train_df_final.csv")
train_df = raw_train.copy()
raw_test = pd.read_csv("test_df_final.csv")
test_df = raw_test.copy()

train_obj = pd.read_csv("train_obj_toInt.csv")
test_obj = pd.read_csv("test_obj_toInt.csv")

#### Kalan Bazı Null Değerlerin doldurulması ve train, test verilerinin hazır hale getirilmesi

In [3]:
columns =list(train_obj.columns.values)

In [4]:
num_train = train_df.drop(columns, axis=1).copy()

In [5]:
train_df.LotFrontage.fillna(0, inplace=True)
num_train.LotFrontage.fillna(0, inplace=True)

In [720]:
train_df[train_df.GarageYrBlt.isnull() == True][['GarageArea']].sum()

GarageArea    0
dtype: int64

In [6]:
train_df.GarageYrBlt.fillna(0, inplace=True)
num_train.GarageYrBlt.fillna(0, inplace=True)

In [722]:
train_df[train_df.MasVnrArea.isnull() == True][['MasVnrType']]

,MasVnrType
234,0
529,0
650,0
936,0
973,0
977,0
1243,0
1278,0


In [7]:
train_df.MasVnrArea.fillna(0, inplace=True)
num_train.MasVnrArea.fillna(0, inplace=True)

In [8]:
num_col = list(num_train.columns.values)

In [9]:
train_df.drop(num_col, axis=1, inplace=True)

In [10]:
train_df = pd.concat([train_df,num_train], axis=1)

In [727]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Columns: 119 entries, MSZoning to SalePrice
dtypes: float64(3), int64(116)
memory usage: 1.3 MB


In [11]:
test_df.drop(['MasVnrType.1'], axis=1, inplace=True)
columns_test =list(test_obj.columns.values)

In [12]:
num_test = test_df.drop(columns_test, axis=1).copy()

In [13]:
train_df.fillna(0, inplace=True)
num_train.fillna(0, inplace=True)

In [ ]:
num_test = test_df.drop(columns, axis=1).copy()
test_df.fillna(0, inplace=True)
num_test.fillna(0, inplace=True)

In [732]:
from sklearn.preprocessing import StandardScaler

In [733]:
X = train_df.iloc[:, :-1]
y = train_df.iloc[:, -1:]

In [734]:
sc_X = StandardScaler()
sc_y = StandardScaler()
X_s = sc_X.fit_transform(X)
y_s = sc_y.fit_transform(y)

In [ ]:
X_test = test_df.copy()
sc_test = StandardScaler()
X_test_s = sc_X.fit_transform(X_test)

#### SVR

In [735]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_s, y_s)

SVR()

In [739]:
y_pred = regressor.predict(X_test_s)
y_pred = sc_y.inverse_transform(y_pred) 

In [740]:
prediction = pd.DataFrame(data=range(1461, 2920), columns=['Id'])

In [741]:
prediction['SalePrice'] = y_pred

In [743]:
prediction.to_csv('prediction_svr.csv', index=False)

In [744]:
#train=>X_s, y_s, test=>X_test

#### XGB Regressor

In [745]:
import xgboost as xgb

In [746]:
xgbr = xgb.XGBRegressor(verbosity=2) 

In [747]:
xgbr.fit(X, y)

[05:24:05] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=6
[05:24:05] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 64 extra nodes, 0 pruned nodes, max_depth=6
[05:24:05] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 76 extra nodes, 0 pruned nodes, max_depth=6
[05:24:05] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 76 extra nodes, 0 pruned nodes, max_depth=6
[05:24:05] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=6
[05:24:05] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 82 extra nodes, 0 pruned nodes, max_depth=6
[05:24:05] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 96 extra nodes, 0 pruned nodes, max_depth=6
[05:24:05] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 98 extra nodes, 0 pruned nodes, max_depth=6
[05:24:05] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 82 extra nodes, 0 pruned no

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=2)

In [748]:
y_pred = xgbr.predict(X_test)

In [749]:
prediction['SalePrice'] = y_pred

In [750]:
prediction

,Id,SalePrice
0,1461,67327.515625
1,1462,55608.152344
2,1463,75804.835938
3,1464,57235.410156
4,1465,49077.066406
...,...,...
1454,2915,47583.742188
1455,2916,46886.394531
1456,2917,78573.179688
1457,2918,69424.351562


#### SVR with Grid Search

In [751]:
prediction.to_csv('prediction_xgbr.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
       'degree': np.arange(1,7),
       'C': [0.1, 1, 10, 100, 1000]}
regressor = SVR()

svr_cv = GridSearchCV(regressor, grid, cv=10)
svr_cv.fit(X_s, y_s, ver)

In [ ]:
y_pred = svr_cv.predict(X_test_s)
y_pred = sc_y.inverse_transform(y_pred) 